In [15]:
### PYTHON IMPORTS
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from random import shuffle
import librosa



In [37]:
""" Convolutional Neural Network.

Build and train a convolutional neural network with TensorFlow.
This example is using the MNIST database of handwritten digits
(http://yann.lecun.com/exdb/mnist/)

Author: Aymeric Damien
Project: https://github.com/aymericdamien/TensorFlow-Examples/
"""

from __future__ import division, print_function, absolute_import
import os
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

# Training Parameters
learning_rate = 0.001
num_steps = 200
batch_size = 128
display_step = 10
# parameters

num_classes = 10 #numberOfClasses

num_input = 128*44*1

# saver train parameters
useCkpt = False
checkpoint_step = 5
checkpoint_dir = os.getcwd()+'\\checkpoint\\'


# Network Parameters
num_input = 28*28 # MNIST data input (img shape: 28*28)
#num_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75 # Dropout, probability to keep units

# tf Graph input
X = tf.placeholder(tf.float32, [None, num_input])
Y = tf.placeholder(tf.float32, [None, num_classes])
keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)


# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    print("conv2")
    print(conv2.shape) # (?, 7, 7, 64)
    
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    print("conv3")
    print(conv3.shape) # (?, 7, 7, 64)
    
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    print("conv4")
    print(conv4.shape) # (?, 7, 7, 64)
    
    conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    conv5 = maxpool2d(conv5, k=2)
    print("conv5")
    print(conv5.shape) # (?, 7, 7, 64)
    
    return conv5
    

def mlp(convFinal, weights, biases, dropout):
    #(?, 3136)
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(convFinal, [-1, weights['wd1'].get_shape().as_list()[0]])
    print("fc1")
    print(fc1.shape)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

def elm(convFinal, weights, biases, dropout):
    #conv5Shape = convFinal.get_shape().as_list()
    conv5Shape = convFinal.get_shape().as_list()
    print("CONC")
    print(conv5Shape)

    # Input to reshape is a tensor with 1280 values, but the requested shape requires a multiple of 512
    print("ELM")
    print(convFinal.shape) # 10
    fc1 = tf.reshape(convFinal, [-1, weights['elm_wd1'].get_shape().as_list()[0]])
    print(fc1.shape) # 512

    fc1 = tf.add(tf.matmul(fc1,weights['elm_wd1']), biases['elm_bd1'])
    print(fc1.shape) # 256

    fc1 = tf.nn.relu(fc1)
    print(fc1.shape) # 256

    return fc1

def elm_prediction(convFinal, weights, biases, dropout):
    #conv5Shape = convFinal.get_shape().as_list()
    
    fc1 = tf.reshape(convFinal, [-1, weights['elm_wd1'].get_shape().as_list()[0]]) #conv5Shape[1] * conv5Shape[2] * conv5Shape[3]])
    fc1 = tf.add(tf.matmul(fc1,weights['elm_wd1']), biases['elm_bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)
    out = tf.add(tf.matmul(fc1,weights['elm_out']), biases['elm_out'])

    prediction = tf.nn.softmax(out)
    
    return prediction

# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    # 5x5 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    
    'wc3': tf.Variable(tf.random_normal([5, 5, 64, 128])),
    
    'wc4': tf.Variable(tf.random_normal([5, 5, 128, 256])),
    
    'wc5': tf.Variable(tf.random_normal([5, 5, 256, 512])),
    # fully connected, 7*7*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([512, 256])),
    'elm_wd1': tf.Variable(tf.random_normal([512, 256])),

    # 1024 inputs, 10 outputs (class prediction)
    'elm_out': tf.Variable(tf.random_normal([256, num_classes])),
    'out': tf.Variable(tf.random_normal([256, num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bc3': tf.Variable(tf.random_normal([128])),
    'bc4': tf.Variable(tf.random_normal([256])),
    'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([256])),
    'elm_bd1': tf.Variable(tf.random_normal([256])),

    'out': tf.Variable(tf.random_normal([num_classes])),
    'elm_out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
mlp_conv = conv_net(X, weights, biases, keep_prob)
logits = mlp(mlp_conv, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

#ELM
elm_convFinal = conv_net(X,weights,biases,keep_prob)

elm_fc1 = elm(elm_convFinal, weights, biases, dropout)
output_weights = tf.matrix_solve_ls(elm_fc1,Y, l2_regularizer=0.0,fast=False,name=None)
weights_output = tf.assign( weights['elm_out'], output_weights, validate_shape=None, use_locking=None, name=None)
elm_logits = elm_prediction(elm_convFinal,weights,biases, dropout)

# evaluate model
correct_pred_elm = tf.equal(tf.argmax(elm_logits,1),tf.argmax(Y,1))
accuracy_elm = tf.reduce_mean(tf.cast(correct_pred_elm,tf.float32))

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()



Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
conv2
(?, 7, 7, 64)
conv3
(?, 4, 4, 128)
conv4
(?, 2, 2, 256)
conv5
(?, 1, 1, 512)
fc1
(?, 512)
conv2
(?, 7, 7, 64)
conv3
(?, 4, 4, 128)
conv4
(?, 2, 2, 256)
conv5
(?, 1, 1, 512)
CONC
[None, 1, 1, 512]
ELM
(?, 1, 1, 512)
(?, 512)
(?, 256)
(?, 256)


In [53]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    for step in range(1, 100+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        
        # Run optimization op (backprop)
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y, keep_prob: 0.8})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y,
                                                                 keep_prob: 1.0})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))

            print("Testing Accuracy:", \
            sess.run(accuracy, feed_dict={X: mnist.test.images[:256],
                                          Y: mnist.test.labels[:256],
                                          keep_prob: 1.0}))
            
    print("Optimization Finished! - Do ELM")
    #Performance before optimization
    accVal = sess.run(accuracy_elm, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1})
    print("Accuracy Before optimizing = " + "{:.4f}".format(accVal))
    # train the model
    sess.run([weights_output], feed_dict={X: mnist.train.images[:30000],Y: mnist.train.labels[:30000], keep_prob: 1}) # Defines the input
    print("Optimization finish!")

    #Performance
    accVal = sess.run(accuracy_elm, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1})
    print("Accuracy after optimizing = " + "{:.4f}".format(accVal))
    print("ELM DONE")
    # Calculate accuracy for 256 MNIST test images
        

Step 1, Minibatch Loss= 611958464.0000, Training Accuracy= 0.055
Testing Accuracy: 0.0703125
Step 10, Minibatch Loss= 161315392.0000, Training Accuracy= 0.305
Testing Accuracy: 0.328125
Step 20, Minibatch Loss= 70898816.0000, Training Accuracy= 0.406
Testing Accuracy: 0.464844
Step 30, Minibatch Loss= 47232552.0000, Training Accuracy= 0.562
Testing Accuracy: 0.605469
Step 40, Minibatch Loss= 30258860.0000, Training Accuracy= 0.641
Testing Accuracy: 0.664063
Step 50, Minibatch Loss= 20646546.0000, Training Accuracy= 0.711
Testing Accuracy: 0.765625
Step 60, Minibatch Loss= 12051636.0000, Training Accuracy= 0.773
Testing Accuracy: 0.765625
Step 70, Minibatch Loss= 14923038.0000, Training Accuracy= 0.805
Testing Accuracy: 0.816406
Step 80, Minibatch Loss= 10181824.0000, Training Accuracy= 0.828
Testing Accuracy: 0.859375
Step 90, Minibatch Loss= 7677191.0000, Training Accuracy= 0.828
Testing Accuracy: 0.867188
Step 100, Minibatch Loss= 11385781.0000, Training Accuracy= 0.844
Testing Accur

In [52]:
# Start training
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    
            
    print("Optimization Finished! - Do ELM")
    #Performance before optimization
    accVal = sess.run(accuracy_elm, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1})
    print("Accuracy Before optimizing = " + "{:.4f}".format(accVal))
    # train the model
    sess.run([weights_output], feed_dict={X: mnist.train.images[:30000],Y: mnist.train.labels[:30000], keep_prob: 1}) # Defines the input
    print("Optimization finish!")

    #Performance
    accVal = sess.run(accuracy_elm, feed_dict={X: mnist.test.images[:256], Y: mnist.test.labels[:256], keep_prob: 1})
    print("Accuracy after optimizing = " + "{:.4f}".format(accVal))
    print("ELM DONE")
    # Calculate accuracy for 256 MNIST test images
        

Optimization Finished! - Do ELM
Accuracy Before optimizing = 0.1367
Optimization finish!
Accuracy after optimizing = 0.1719
ELM DONE
